# Авиарейсы без потерь

### Предобработка данных :

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df = pd.read_csv('query_result_2021-12-29T05_21_17.69909Z.csv')

## По данным официального сайта https://favt.gov.ru расход топлива самолетов SSJ-100 составляет 1700 кг/час, 
##    а В737-300 составляет 2400 кг/час.
## Добавим колонку 'fuel_consumption' с этими данными для расчета расхода топлива за рейс .Расход топлива за рейс 
## выведем в колонке 'total_fuel_consumption' 

fuel_consumption = []
for i in df['flight_no']:
    if i =='PG0480':
        fuel_consumption.append(int('1700'))
    elif i =='PG0252':
        fuel_consumption.append(int('2400'))
df['fuel_consumption'] = fuel_consumption
        
total_fuel_consumption = []
for i in range(0, len(df)):
    total_fuel = df.time_fl[i] * df.fuel_consumption[i]
    total_fuel_consumption.append(abs(total_fuel))
df['total_fuel_consumption'] = total_fuel_consumption

## По данным этого же сайта цена на топливо составляла:
##    декабрь 2016г. - 38867,00 руб/тонна
##    январь  2017г. - 41435,00 руб/тонна
##    февраль 2017г. - 39553,00 руб/тонна
    
## Исходя из этих данных вычислим затраты на топливо для каждого рейса и выведем в колонке 'fuel_amount'
## но перед этим преобразуем формат даты в колонке 'actual_departure' чтобы обращаться к месяцам напрямую:

datetime_list=[]
for date in df.actual_departure:
    dt=datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
    datetime_list.append(dt)
df['actual_departure'] = datetime_list

fuel_am = []
for x in range(0, len(df)):
    if df['actual_departure'][x].month == 12:        ## декабрь 2016г.
        a = df.total_fuel_consumption[x] * 38.867
        a = round(a, 2)
        fuel_am.append(a)
    elif df['actual_departure'][x].month == 1:       ## январь 2017г.
        a = df.total_fuel_consumption[x] * 41.435
        a = round(a, 2)
        fuel_am.append(a)
    elif df['actual_departure'][x].month == 2:       ## февраль 2017г.
        a = df.total_fuel_consumption[x] * 39.553
        a = round(a, 2)
        fuel_am.append(a)
df['fuel_amount'] = fuel_am

## Посчитаем разницу между выручкой за билеты и расходами на топливо для рейса,
## результаты выведем в колонке "profit"  :

profit = []
for x in range(0, len(df)):
    total = df.total_amount[x] - df.fuel_amount[x]
    profit.append(total)
df['profit'] = profit


## Выведем получившийся датасет

display(df)

,flight_id,flight_no,model,departure_airport,arrival_airport,actual_departure,count_ticket,count_seat_no,time_fl,total_amount,fuel_consumption,total_fuel_consumption,fuel_amount,profit
0,136698,PG0480,Sukhoi Superjet-100,AAQ,EGO,2016-12-01 09:30:00,94,94,0.816667,746400.0,1700,1388.333333,53960.35,692439.65
1,136774,PG0480,Sukhoi Superjet-100,AAQ,EGO,2016-12-02 09:27:00,93,93,0.833333,740100.0,1700,1416.666667,55061.58,685038.42
2,136873,PG0480,Sukhoi Superjet-100,AAQ,EGO,2016-12-03 09:26:00,96,96,0.850000,759000.0,1700,1445.000000,56162.81,702837.19
3,136843,PG0480,Sukhoi Superjet-100,AAQ,EGO,2016-12-04 09:30:00,78,78,0.833333,632400.0,1700,1416.666667,55061.58,577338.42
4,136943,PG0480,Sukhoi Superjet-100,AAQ,EGO,2016-12-05 09:28:00,94,94,0.833333,733800.0,1700,1416.666667,55061.58,678738.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,136120,PG0252,Boeing 737-300,AAQ,SVO,2017-02-24 10:07:00,109,109,1.650000,1605400.0,2400,3960.000000,156629.88,1448770.12
176,136439,PG0252,Boeing 737-300,AAQ,SVO,2017-02-25 10:08:00,117,117,1.666667,1701800.0,2400,4000.000000,158212.00,1543588.00
177,136351,PG0252,Boeing 737-300,AAQ,SVO,2017-02-26 10:13:00,105,105,1.666667,1555400.0,2400,4000.000000,158212.00,1397188.00
178,136266,PG0252,Boeing 737-300,AAQ,SVO,2017-02-27 10:08:00,108,108,1.666667,1566400.0,2400,4000.000000,158212.00,1408188.00


Для подготовки презентации и анализа по прибыльности рейса сохраним датасет в файл "0_project_4.csv" и 
в дальнейшем будем использовать его

In [36]:
df.to_csv("0_project_4.csv", index = True, sep=',', encoding='utf-8')